In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from scipy.stats import spearmanr
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [2]:
train = pd.read_csv('TON-Iot Dataset Train_Test Network/Train_Test_Network.csv')

In [3]:
train['type'].value_counts()

train.drop(['dst_pkts','src_pkts','dns_RA','dns_RD','dns_qclass','dns_qtype','dns_rejected','dns_query','http_version','http_uri','http_user_agent','http_request_body_len','http_response_body_len','http_status_code'], axis=1, inplace=True)

In [4]:
# df = pd.concat([train,test]).reset_index(drop=True)
df = train.copy()
cols_cat = df.select_dtypes('object').columns # To be explained later
cols_numeric = df._get_numeric_data().columns # To be explained later

In [5]:
def Remove_dump_values(df, cols):
    for col in cols:
        df[col] = np.where(df[col] == '-', 'None', df[col])
    return df

In [6]:
cols = df.columns
data_bin = Remove_dump_values(df, cols)
data_bin = data_bin.drop(['src_ip','dst_ip','ts'], axis=1)
data_bin_attackcat = data_bin[['type']]

In [7]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories='auto')
oe.fit(data_bin_attackcat)
data_bin_attackcat = oe.transform(data_bin_attackcat)

In [8]:
data_bin.drop(['type'], axis=1, inplace=True)
cols_cat = cols_cat.drop(['type','src_ip','dst_ip'])
data_bin_hot = pd.get_dummies(data_bin,columns=cols_cat)
data_bin_hot

,src_port,dst_port,duration,src_bytes,dst_bytes,missed_bytes,src_ip_bytes,dst_ip_bytes,dns_rcode,label,...,weird_name_data_before_established,weird_name_dnp3_corrupt_header_checksum,weird_name_inappropriate_FIN,weird_name_possible_split_routing,weird_addl_43,weird_addl_46,weird_addl_48,weird_addl_None,weird_notice_F,weird_notice_None
0,1883,52976,80549.53026,1762852,41933215,0,14911156,236,0,0,...,False,False,False,False,False,False,False,True,True,False
1,47260,15600,0.0,0,0,0,63,0,0,0,...,False,False,False,False,False,False,False,True,False,True
2,1880,51782,0.0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,True,False
3,34296,10502,0.0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,True
4,46608,53,0.000549,0,298,0,0,354,0,0,...,False,False,False,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,33108,80,0.0,0,0,0,60,0,0,1,...,False,False,False,False,False,False,False,True,False,True
461039,37242,443,0.0,0,0,0,0,103,0,1,...,False,False,False,False,False,False,False,True,False,True
461040,4444,49178,290.371539,101568,2592,0,108064,3832,0,1,...,False,False,False,False,False,False,False,True,False,True
461041,60816,443,23.190902,32,31,0,411,395,0,1,...,False,False,False,False,False,False,False,True,False,True


In [9]:
# print value count of all columns
for col in train.columns:
    print(train[col].value_counts())

ts
1556423502    1194
1556340865     976
1556423501     953
1556423638     934
1556340867     776
              ... 
1554283049       1
1554283045       1
1554283044       1
1554283043       1
1556436603       1
Name: count, Length: 115666, dtype: int64
src_ip
192.168.1.190     77033
192.168.1.152     67784
192.168.1.30      61637
192.168.1.79      41098
192.168.1.195     36089
                  ...  
255.42.60.52          1
243.111.49.216        1
10.3.191.169          1
74.166.95.98          1
118.164.23.26         1
Name: count, Length: 11536, dtype: int64
src_port
1880     18631
43530    17313
42100    16688
34296    15967
53972     8354
         ...  
2632         1
31772        1
4322         1
6429         1
44197        1
Name: count, Length: 53671, dtype: int64
dst_ip
192.168.1.190    80102
192.168.1.152    60862
192.168.1.255    24295
192.168.1.195    23177
192.168.1.193    22902
                 ...  
200.62.191.10        1
192.195.66.12        1
85.90.64.164         1
202.3

In [28]:
cols_cat

Index(['proto', 'service', 'conn_state', 'dns_AA', 'ssl_version', 'ssl_cipher',
       'ssl_resumed', 'ssl_established', 'ssl_subject', 'ssl_issuer',
       'http_trans_depth', 'http_method', 'http_orig_mime_types',
       'http_resp_mime_types', 'weird_name', 'weird_addl', 'weird_notice'],
      dtype='object')

In [10]:
attack_cat_df = pd.DataFrame(data_bin_attackcat, columns=['type'])

In [11]:
data_bin_hot = data_bin_hot.reset_index(drop=True)
data_bin_hot2 = pd.concat([data_bin_hot,attack_cat_df],axis=1, join='inner',ignore_index=False)

In [12]:
cols_numeric = list(cols_numeric)
cols_numeric.remove('label')
cols_numeric.remove('ts')

In [13]:
data_bin_hot2[cols_numeric] = data_bin_hot2[cols_numeric].astype('float') 
data_bin_hot2[cols_numeric] = (data_bin_hot2[cols_numeric] - np.min(data_bin_hot2[cols_numeric])) / np.std(data_bin_hot2[cols_numeric])

In [33]:
data_bin_hot2

,src_port,dst_port,duration,src_bytes,dst_bytes,missed_bytes,src_ip_bytes,dst_ip_bytes,dns_rcode,label,...,weird_name_dnp3_corrupt_header_checksum,weird_name_inappropriate_FIN,weird_name_possible_split_routing,weird_addl_43,weird_addl_46,weird_addl_48,weird_addl_None,weird_notice_F,weird_notice_None,type
0,0.101952,3.799303,175.726962,0.152327,3.438007,0.0,86.672008,0.001470,0.0,0,...,False,False,False,False,False,False,True,True,False,5.0
1,2.558822,1.118792,0.000000,0.000000,0.000000,0.0,0.000366,0.000000,0.0,0,...,False,False,False,False,False,False,True,False,True,5.0
2,0.101790,3.713672,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0,...,False,False,False,False,False,False,True,True,False,5.0
3,1.856906,0.753177,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0,...,False,False,False,False,False,False,True,False,True,5.0
4,2.523521,0.003801,0.000001,0.000000,0.000024,0.0,0.000000,0.002205,0.0,0,...,False,False,False,False,False,False,True,True,False,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1.792583,0.005737,0.000000,0.000000,0.000000,0.0,0.000349,0.000000,0.0,1,...,False,False,False,False,False,False,True,False,True,9.0
461039,2.016413,0.031771,0.000000,0.000000,0.000000,0.0,0.000000,0.000642,0.0,1,...,False,False,False,False,False,False,True,False,True,7.0
461040,0.240614,3.526920,0.633475,0.008776,0.000213,0.0,0.628129,0.023869,0.0,1,...,False,False,False,False,False,False,True,False,True,0.0
461041,3.292792,0.031771,0.050593,0.000003,0.000003,0.0,0.002389,0.002460,0.0,1,...,False,False,False,False,False,False,True,False,True,4.0


In [15]:
# oversample minority types, which are all attack types other then normal using SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X = data_bin_hot2.drop('type', axis=1)
y = data_bin_hot2['type']
X_res, y_res = sm.fit_resample(X, y)

# convert to dataframe
X_res = pd.DataFrame(X_res, columns=X.columns)
y_res = pd.DataFrame(y_res, columns=['type'])

# concatenate X_res and y_res
data_balanced = pd.concat([X_res, y_res], axis=1)

# check new class counts
data_balanced['type'].value_counts()

type
5.0    300000
8.0    300000
2.0    300000
3.0    300000
1.0    300000
6.0    300000
9.0    300000
7.0    300000
0.0    300000
4.0    300000
Name: count, dtype: int64

In [44]:
# save balanced data to csv
data_balanced.to_csv('ToN-IoT_balanced_smote.csv', index=False)